In [1]:
from fastai.tabular.all import *
from sklearn.model_selection import KFold, GroupKFold
from optuna.integration import FastAIPruningCallback
import optuna

In [2]:
STOCK_COUNT = 112
FEATURE_COUNT = 240#20


In [3]:
def fill_missing(train_df):
    all_times = train_df.time_id.unique()
    all_stocks = train_df.stock_id.unique()
    filled_df = train_df.copy()
    filled_df=filled_df.set_index(['time_id', 'stock_id'])
    new_index = pd.MultiIndex.from_product([all_times, all_stocks], names = ['time_id', 'stock_id'])
    filled_df = filled_df.reindex(new_index).reset_index()
    filled_df = filled_df.fillna(0)
    return filled_df

In [4]:
class MaskTfm(ItemTransform):
    do_tranform=0
    def mask(self, x, indices):
        x[torch.tensor(indices, device=x.device)] = 0
        return x
    
    def __call__(self, b, split_idx=None, **kwargs):
        self.do_transform = (split_idx == 0)
        return super().__call__(b, split_idx=split_idx, **kwargs)
    

    def encodes(self, x):
        if not self.do_transform: return x
        n = len(x[0])
        indices = np.random.choice(np.array(range(n)), n//10, replace=False)
        x = [self.mask(y, indices) for y in x]
        
        return x

class MyDataLoader(TabDataLoader):

        
    def shuffle_fn(self, idxs):
        idxs = np.array(idxs).reshape(-1,112)
        np.random.shuffle(idxs)
        return idxs.reshape(-1).tolist()
    
    def __init__(self, dataset, bs=16, shuffle=False, after_batch=None, num_workers=0, **kwargs):
        if after_batch is None: after_batch = L(TransformBlock().batch_tfms)+ReadTabBatch(dataset) + [MaskTfm()]
        super().__init__(dataset, bs=bs, shuffle=shuffle, after_batch=after_batch, num_workers=num_workers, **kwargs)

    



def get_dls(train_df, bs, trn_idx, val_idx):
    cont_nn,cat_nn = cont_cat_split(train_df, max_card=9000, dep_var='target')
    cat_nn=[x for x in cat_nn if not x in ['row_id', 'time_id']]
    
    procs_nn = [Categorify, Normalize]
    to_nn = TabularPandas(train_df, procs_nn, cat_nn, cont_nn, splits=[list(trn_idx), list(val_idx)], y_names='target')
    dls = to_nn.dataloaders(bs=112*100, shuffle=True, dl_type = MyDataLoader)
    dls.train_ds.split_idx=0
    dls.valid_ds.split_idx=1
    return dls

In [5]:
train_df = pd.read_csv('train_with_features.csv')

In [6]:
#train_df = pd.read_feather('train_24cols.feather')
train_df = fill_missing(train_df)
trn_idx0, val_idx0 = first(GroupKFold().split(train_df, groups = train_df.time_id))

In [7]:
dls = get_dls(train_df, 100, trn_idx0, val_idx0)

In [19]:
class TimeEncoding(nn.Module):
    def __init__(self, inp_size, bottleneck, p, multiplier):
        super().__init__()
        self.multiplier  = multiplier#nn.Parameter(torch.tensor(multiplier)) 
        self.initial_layers = LinBnDrop(inp_size, bottleneck, act=nn.ReLU(True), p=p, bn=False)
        
        self.concat_layers = nn.Sequential(
            nn.Linear(bottleneck * STOCK_COUNT, inp_size),
            nn.Tanh()
        )
        
    def forward(self, x):
        y = self.initial_layers(x)
        times = y.shape[0] // STOCK_COUNT
        y = y.view(times, -1)
        y = self.concat_layers(y)
   
        y = y.view(times,1,-1).expand(times,STOCK_COUNT,-1).contiguous().view(times*STOCK_COUNT, -1)
        
        return x + y * self.multiplier

class BN(nn.Module):
    def __init__(self, features):
        super().__init__()
        self.num_features = features
        self.bn = nn.BatchNorm1d(STOCK_COUNT * self.num_features)
    def forward(self, x):
        sh = x.shape
        x = x.view(-1, STOCK_COUNT * self.num_features)
        x = self.bn(x)
        return x.view(*sh)
    
class ParallelModel(nn.Module):
    def __init__(self, inp_size, emb_size, lin_sizes, ps, bottleneck, time_ps, multiplier ):
        super().__init__()
        
        self.stock_emb = nn.Parameter(torch.empty(STOCK_COUNT, emb_size))
        nn.init.normal_(self.stock_emb)
        
        lin_sizes = [inp_size+emb_size] + lin_sizes
        layers = []
        for n_in, n_out, p, time_p in zip(lin_sizes, lin_sizes[1:], ps, time_ps):
            layers.append(BN(n_in ))
            if p: layers.append(nn.Dropout(p))
            layers.append(nn.Linear(n_in, n_out))
            layers.append(nn.ReLU(True))
            
            layers.append(TimeEncoding(n_out, bottleneck, time_p, multiplier))
        layers.append(LinBnDrop(lin_sizes[-1], 1, bn=False))
        layers.append(SigmoidRange(0, .1))
        self.layers = nn.Sequential(*layers)
    
    
    def forward(self, x_cat, x_cont):
        times = x_cat.shape[0] // STOCK_COUNT
        s_e = self.stock_emb.repeat(times, 1)
        x = torch.cat([x_cont, s_e], dim=1)
        mx = x.max()
        for l in self.layers.children():
            #print(x.shape, x.mean(), x.std())
            x = l(x)
            mx = x.max()
        if mx > 10:print(mx)
        return x#self.layers(x)

In [9]:

def rmspe(preds, targs):
    mask = targs != 0
    targs, preds = torch.masked_select(targs, mask), torch.masked_select(preds, mask)
    x = (targs-preds)/targs
    res = (x**2).mean().sqrt()
    if torch.isnan(res): 
        print(targs)
        print(preds)
        raise Exception('fck loss is nan')
    return res

In [10]:
def train(trial, dls, save_as = None):
    inp_size = FEATURE_COUNT
    emb_size = trial.suggest_int('emb_size', 3, 30)
    max_sizes = [2000, 1000, 500]
    lin_sizes = [trial.suggest_int(f'lin_size{i}', 10, ms) for i, ms in enumerate(max_sizes)]
    ps = [0]+[trial.suggest_float(f'p{i}', 0, .8) for i in range(1,3)]
    
    bottleneck = trial.suggest_int('bottleneck', 5, 100)
    time_ps = [trial.suggest_float(f'time_p{i}', 0, .5) for i in range(3)]
    multiplier = trial.suggest_float('multiplier', .01, .5)
    lr = float(trial.suggest_float('lr', 1e-3, 1e-2))
    
    model = ParallelModel(inp_size, emb_size, lin_sizes, ps, bottleneck, time_ps, multiplier)
    #bx1, bx2, by = dls.one_batch()
    
    learn = Learner(dls,model = model, loss_func=rmspe, metrics=AccumMetric(rmspe), opt_func=ranger).to_fp16()
    # with learn.no_bar():
    #     with learn.no_logging():    
    learn.fit_flat_cos(50, lr)
    #with learn.no_logging(): 
    if save_as:
        learn.save(save_as)
    return L(learn.recorder.values).itemgot(2)[-1]

In [11]:
def train_cross_valid(trial, dlss, save_as=None):
    res = 0
    for idx, dls in enumerate(dlss):
        v = train(trial, dls, save_as + str(idx) if save_as else None)
        print(f'fold {idx}: {v}')
        res +=v;
    return res/5

In [ ]:
study = optuna.load_study('parallel4','sqlite:///optuna.db' )

In [ ]:
study = optuna.create_study(direction="minimize", study_name = 'parallel_fixed_mult', storage='sqlite:///optuna.db', load_if_exists=True, pruner=optuna.pruners.NopPruner(), sampler=None)
study.optimize(functools.partial(train, dls=dls))

[I 2021-09-20 21:17:06,409] Using an existing study with name 'parallel_fixed_mult' instead of creating a new one.


epoch,train_loss,valid_loss,rmspe,time
0,19.878239,20.145290,20.158358,00:00
1,18.255226,16.074665,16.084969,00:00
2,15.925702,12.088929,12.096025,00:00
3,13.254589,8.724669,8.728976,00:00
4,10.691133,6.529365,6.532411,00:00
5,8.431808,4.772106,4.774443,00:00
6,6.540692,3.407113,3.408715,00:00
7,4.987409,2.347702,2.348614,00:00
8,3.728291,1.565269,1.565905,00:00
9,2.733438,0.997414,0.997912,00:00


[I 2021-09-20 21:17:50,406] Trial 45 finished with value: 0.25317320227622986 and parameters: {'emb_size': 16, 'lin_size0': 827, 'lin_size1': 338, 'lin_size2': 356, 'p1': 0.35934634877626714, 'p2': 0.21500230229686917, 'bottleneck': 60, 'time_p0': 0.17240412789238566, 'time_p1': 0.2722105194052167, 'time_p2': 0.26333635094558483, 'multiplier': 0.4637178722284131, 'lr': 0.007690704414846785}. Best is trial 29 with value: 0.2165665328502655.


epoch,train_loss,valid_loss,rmspe,time
0,20.839403,20.400064,20.412752,00:00
1,18.643988,14.968852,14.977967,00:00
2,15.449907,9.412355,9.417967,00:00
3,11.891209,5.279018,5.282197,00:00
4,8.630199,2.907056,2.909145,00:00
5,6.016031,1.548570,1.550181,00:00
6,4.070316,0.809631,0.813076,00:00
7,2.691597,0.424722,0.426809,00:00
8,1.745035,0.333833,0.334150,00:00
9,1.138657,0.382207,0.382249,00:00


[I 2021-09-20 21:18:37,414] Trial 46 finished with value: 0.2678394019603729 and parameters: {'emb_size': 19, 'lin_size0': 673, 'lin_size1': 885, 'lin_size2': 453, 'p1': 0.7191184773219781, 'p2': 0.17295930968778012, 'bottleneck': 48, 'time_p0': 0.11390485055951897, 'time_p1': 0.16906075624925493, 'time_p2': 0.16939931523115287, 'multiplier': 0.40404629316755425, 'lr': 0.009513754112396525}. Best is trial 29 with value: 0.2165665328502655.


...

epoch,train_loss,valid_loss,rmspe,time
0,22.544031,20.842684,20.854330,00:00
1,19.470615,14.570962,14.577313,00:00
2,15.588395,9.602162,9.605283,00:00
3,11.905502,6.275138,6.277641,00:00
4,8.813865,4.137307,4.139039,00:00
5,6.283940,2.463076,2.464658,00:00
6,4.336133,1.375065,1.376637,00:00
7,2.910421,0.740256,0.743449,00:00
8,1.908007,0.426296,0.431745,00:00
9,1.236368,0.304072,0.306914,00:00


[I 2021-09-20 21:36:58,757] Trial 69 finished with value: 0.249508798122406 and parameters: {'emb_size': 14, 'lin_size0': 1608, 'lin_size1': 757, 'lin_size2': 126, 'p1': 0.03867447696991147, 'p2': 0.21599836519530877, 'bottleneck': 34, 'time_p0': 0.07457624509561464, 'time_p1': 0.1530173311229674, 'time_p2': 0.17239285964670986, 'multiplier': 0.4284300273005639, 'lr': 0.008021842086017387}. Best is trial 29 with value: 0.2165665328502655.


epoch,train_loss,valid_loss,rmspe,time
0,21.309282,20.990568,21.003408,00:00
1,19.582909,17.007355,17.017729,00:00
2,16.903507,12.508971,12.516782,00:00
3,13.792915,8.788277,8.793698,00:00
4,10.788732,6.145353,6.149113,00:00
5,8.216824,4.236410,4.239372,00:00
6,6.109562,2.848700,2.850686,00:00
7,4.438314,1.872426,1.873722,00:00
8,3.166567,1.244431,1.245361,00:00
9,2.225968,0.802325,0.802968,00:00


[I 2021-09-20 21:37:44,858] Trial 70 finished with value: 0.22928987443447113 and parameters: {'emb_size': 23, 'lin_size0': 1108, 'lin_size1': 936, 'lin_size2': 29, 'p1': 0.20296675911148818, 'p2': 0.26120106074909955, 'bottleneck': 38, 'time_p0': 0.23411922578253758, 'time_p1': 0.4679781919934001, 'time_p2': 0.3543650084912808, 'multiplier': 0.058298543741355124, 'lr': 0.007358575227602423}. Best is trial 29 with value: 0.2165665328502655.


epoch,train_loss,valid_loss,rmspe,time
0,20.733707,19.816072,19.826887,00:00
1,18.890039,15.755530,15.762752,00:00
2,16.262896,11.546837,11.551209,00:00
3,13.265852,8.158920,8.161892,00:00
4,10.440565,5.738366,5.740596,00:00
5,8.033134,4.103488,4.105716,00:00
6,6.063903,2.859870,2.861841,00:00


In [14]:
trials = [t for t in study.trials if t.value is not None]

In [15]:
trials =sorted(trials,key = lambda x: x.value)

In [16]:
dlss = [get_dls(train_df,100, trn_idx, val_idx) for trn_idx, val_idx in GroupKFold().split(train_df, groups = train_df.time_id)]

In [17]:
best = study.best_trial

In [21]:
best.params

{'bottleneck': 41,
 'emb_size': 11,
 'lin_size0': 764,
 'lin_size1': 842,
 'lin_size2': 75,
 'lr': 0.006267668552837353,
 'multiplier': 0.18618690920965522,
 'p1': 0.7149990749997216,
 'p2': 0.29950526254816967,
 'time_p0': 0.34702106813252764,
 'time_p1': 0.07144546307698028,
 'time_p2': 0.12147426684839502}

In [25]:
# my_trial = optuna.create_trial(value=42, params=my_params, distributions=best.distributions)

# train_cross_valid(my_trial, dlss)

In [ ]:
for i in range(10):
    trial = trials[i]
    r = train_cross_valid(trial, dlss)
    print('trial', i,':',r)